In [6]:
import cv2
import numpy as np

# Load image
img = cv2.imread('hog.jpg')

In [11]:

def sobel(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Apply Sobel operator in the x and y directions
    sobelx = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
    sobely = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    grad_x = cv2.filter2D(gray, -1, sobelx)
    grad_y = cv2.filter2D(gray, -1, sobely)

    # Compute the gradient magnitude and direction
    magnitude = np.sqrt(np.square(grad_x) + np.square(grad_y))
    angle = np.arctan2(grad_y, grad_x)
    return grad_x,grad_y

In [18]:
# Preprocess image
img = cv2.resize(img, (64, 128))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)

# Compute gradients
grad_x=sobel(img)
grad_y=sobel(img)


In [23]:

import cv2
import numpy as np

# Load the image
img = cv2.imread('hog.jpg')

# Resize the image to maintain aspect ratio
height, width, _ = img.shape
aspect_ratio = 64/128
if height/width > aspect_ratio:
    new_height = int(width * aspect_ratio)
    img = cv2.resize(img, (width, new_height))
else:
    new_width = int(height / aspect_ratio)
    img = cv2.resize(img, (new_width, height))

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Sobel operator to get gradients
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

# Get magnitude and phase
mag, phase = cv2.cartToPolar(sobelx, sobely, angleInDegrees=True)

# Divide the image into cells and get histograms for each cell
cell_size = 8
num_bins = 9
height, width = mag.shape
num_cells_height = height // cell_size
num_cells_width = width // cell_size
histogram = np.zeros((num_cells_height, num_cells_width, num_bins))

for i in range(num_cells_height):
    for j in range(num_cells_width):
        cell_mag = mag[i * cell_size:(i + 1) * cell_size,
                       j * cell_size:(j + 1) * cell_size]
        cell_phase = phase[i * cell_size:(i + 1) * cell_size,
                           j * cell_size:(j + 1) * cell_size]
        histogram[i, j, :], _ = np.histogram(
            cell_phase, bins=num_bins, range=(0, 180), weights=cell_mag)

# Normalize each histogram
eps = 1e-7
histogram = histogram / (np.sum(histogram, axis=2, keepdims=True) + eps)

# Draw HoG ove
r the image
hog_image = np.zeros_like(img)
max_magnitude = np.max(mag)

for i in range(num_cells_height):
    for j in range(num_cells_width):
        cell_img = hog_image[i * cell_size:(i + 1) * cell_size,
                             j * cell_size:(j + 1) * cell_size]
        cell_hist = histogram[i, j, :]
        angle = (cell_hist.argmax() * 20 + 10) % 180
        magnitude = cell_hist.max()
        x = int((i + 0.5) * cell_size)
        y = int((j + 0.5) * cell_size)
        dx = int(magnitude * max_magnitude * np.cos(angle * np.pi)

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_19660/1561160188.py, line 61)

In [2]:
import numpy as np
import cv2


def preprocess_image(image):
    # Resize the image to the appropriate aspect ratio
    resized_image = cv2.resize(image, (64, 128))
    return resized_image


def get_gradients(image):
    # Compute the gradients in the x and y directions using the Sobel kernel
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=1)
    # Compute the magnitude and phase of the gradients
    magnitude, phase = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    return magnitude, phase


def get_histogram(magnitude, phase):
    # Divide the image into 8x8 cells
    cell_size = 8
    num_cells_x = int(magnitude.shape[1] / cell_size)
    num_cells_y = int(magnitude.shape[0] / cell_size)
    # Compute the histogram for each cell based on the directions
    histogram = np.zeros((num_cells_y, num_cells_x, 9))
    for i in range(num_cells_y):
        for j in range(num_cells_x):
            cell_magnitude = magnitude[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]
            cell_phase = phase[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]
            # Divide the directions into 9 bins
            bins = np.linspace(0, 180, 10)
            # Compute the histogram for the cell
            cell_histogram, _ = np.histogram(cell_phase, bins=bins, weights=cell_magnitude)
            histogram[i, j, :] = cell_histogram
    return histogram


def normalize_histograms(histogram):
    # Apply L2-norm to each histogram
    eps = 1e-7
    histogram = histogram / np.sqrt(np.sum(histogram ** 2, axis=-1, keepdims=True) + eps)
    return histogram


def concatenate_histograms(histogram):
    # Concatenate all histograms into a single feature vector
    feature_vector = histogram.ravel()
    return feature_vector


def apply_ratio_histogram(histogram):
    # Apply histogram using the ratio method
    threshold = 0.2
    for i in range(histogram.shape[0]):
        for j in range(histogram.shape[1]):
            # Find the maximum value in the histogram
            max_value = np.max(histogram[i, j, :])
            # Find the index of the maximum value
            max_index = np.argmax(histogram[i, j, :])
            # Check if the maximum value is above the threshold
            if max_value >= threshold:
                # Set all other values to zero except the maximum value
                histogram[i, j, :] = 0
                histogram[i, j, max_index] = max_value
    return histogram


def normalize_block_histograms(histogram):
    # Normalize each 4 histograms together in 16x16 block
    block_size = (16, 16)
    block_stride = (8, 8)
    eps = 1e-7
    # Compute the number of blocks in the image
    num_blocks_x = int((histogram.shape[1] - block_size[1]) / block_stride[1]) + 10

SyntaxError: invalid non-printable character U+00A0 (Temp/ipykernel_17360/2678896994.py, line 75)